In [31]:
import pandas as pd
import geopandas as gpd

import utils.analysis as an

# Zonas Medellin

In [ ]:
zo = ['aguacatala_HL', 'moravia_LH', 'floresta_HH']
# z = zo[0]
for z in zo:
    print(f"Processing zone: {z}")

    zone_edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    zone_nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    sidewalks = gpd.read_file('../data/input/shape/sidewalks/sidewalks.shp')
    zone_edges = zone_edges.to_crs('epsg:32618')
    sidewalks = sidewalks.to_crs('epsg:32618')

    # Create a buffer around the street segments (e.g., 20 meters)
    zone_edges = zone_edges.copy()
    zone_edges['buffer'] = zone_edges['geometry'].buffer(20)  # Adjust buffer size as needed

    # Create a column to store the proportion of sidewalk presence
    zone_edges['sidewalk'] = 0.0

    # Iterate over each buffer and calculate the lengths of sidewalks within it
    for idx, edge in zone_edges.iterrows():
        # Filter the sidewalks contained in the buffer
        sidewalks_in_buffer = sidewalks[sidewalks['geometry'].within(edge['buffer'])]

        # Calculate the total length of sidewalks within the buffer
        total_length = sidewalks_in_buffer['geometry'].length.sum()
        
        # Calculate the length of sidewalks with sidewalk=1
        length_with_sidewalks = sidewalks_in_buffer[sidewalks_in_buffer['sidewalk'] == 1]['geometry'].length.sum()
        
        # Avoid division by zero
        if total_length > 0:
            # Calculate the proportion of sidewalks (length of sidewalks with sidewalk=1 / total length)
            zone_edges.at[idx, 'sidewalk'] = length_with_sidewalks / total_length

    zone_edges = zone_edges.drop(['buffer'], axis=1)
    zone_edges = zone_edges.to_crs('epsg:4326')
    zone_nodes = zone_nodes.to_crs('epsg:4326')
    zone_edges.to_file(f'../data/output/shape/sidewalks/{z}/{z}_sidewalks_edges.shp')
    zone_nodes.to_file(f'../data/output/shape/sidewalks/{z}/{z}_sidewalks_nodes.shp')